In [1]:
import polars as pl
import pandas as pd

# 讀取 parquet
train_filled = pl.read_parquet("data/train_filled_v1_with_veiw.parquet")

# 讀取top50 features
n_top = 80
save_dir = "model_output/selected_features_xgb/one_model/features_v1_with_view"
df = pd.read_csv("model_output/all_features_xgb/v1_add_flight_view/v1_model/model_importance/common_features_with_ranks.csv")
top50_features = df["feature"].head(n_top).tolist()

# 你要保留的欄位 (Top50 + target + group id)
cols_to_keep = top50_features + ["selected", "ranker_id"]

# 只保留這些欄位
train_filled = train_filled.select(cols_to_keep)


print(f"✅ DataFrame 現在只有 {len(train_filled.columns)} 欄位: {train_filled.columns}")


✅ DataFrame 現在只有 82 欄位: ['total_is_min_transfers', 'total_num_transfers_rank', 'price_per_duration_rank', 'price_from_median_zscore', 'price_per_duration', 'legs0_segments0_flightNumber', 'price_percentile', 'price_minus_fee_rank', 'companyID', 'legs0_mean_cabin', 'legs0_max_duration_cabin', 'price_per_fee_rank', 'legs1_mean_cabin', 'both_legs_carrier_all_same', 'days_before_departure', 'legs0_weighted_mean_cabin', 'has_access_tp', 'totalPrice_rank', 'pricingInfo_isAccessTP', 'legs0_arrivalAt_hour', 'legs0_departureAt_hour', 'price_per_tax', 'total_weighted_mean_cabin', 'legs1_max_duration_cabin', 'legs0_segments0_baggageAllowance_quantity', 'price_per_fee', 'leg0_view_diff_mean', 'legs0_segments0_seatsAvailable', 'all_view_diff_mean', 'baggage_total', 'log_price', 'duration_ratio', 'legs1_segments0_flightNumber', 'isVip', 'legs1_segments0_baggageAllowance_quantity', 'price_minus_fee', 'legs1_departureAt_hour', 'total_fees', 'legs1_arrivalAt_hour', 'corporateTariffCode', 'legs1_segment

In [3]:
# 先把 selected 轉成整數 (如果是 bool)
train_filled = train_filled.with_columns(
    pl.col("selected").cast(pl.Int8)
)

# Group by companyID，計算每組selected=1的數量
selected_counts = (
    train_filled
    .group_by("companyID")
    .agg(
        pl.sum("selected").alias("n_selected")
    )
    .filter(pl.col("n_selected") > 1)
)

print(selected_counts)


shape: (575, 2)
┌───────────┬────────────┐
│ companyID ┆ n_selected │
│ ---       ┆ ---        │
│ i64       ┆ i64        │
╞═══════════╪════════════╡
│ 45761     ┆ 204        │
│ 54294     ┆ 33         │
│ 60579     ┆ 73         │
│ 61228     ┆ 73         │
│ 58203     ┆ 49         │
│ …         ┆ …          │
│ 44132     ┆ 4          │
│ 59596     ┆ 3          │
│ 59096     ┆ 825        │
│ 61171     ┆ 3          │
│ 56994     ┆ 4          │
└───────────┴────────────┘


In [2]:
# 要轉成 int 的欄位
cols_to_int = [
    "pricingInfo_isAccessTP",
    "legs0_segments0_baggageAllowance_quantity",
    "legs1_segments0_baggageAllowance_quantity",
    "miniRules1_statusInfos",
    "baggage_total",
    "legs0_segments0_seatsAvailable",
    "miniRules1_monetaryAmount",
    "total_fees",
    "price_minus_fee",
    "taxes",
    "totalPrice",
    "legs1_segments0_seatsAvailable"
]

# 先檢查哪些欄位存在
existing_cols = [c for c in cols_to_int if c in train_filled.columns]

print(f"✅ 共找到 {len(existing_cols)} 個存在的欄位要轉 int: {existing_cols}")

# 做轉型
train_filled = train_filled.with_columns([
    pl.col(c).fill_null(0).cast(pl.Int32).alias(c)
    for c in existing_cols
])

✅ 共找到 7 個存在的欄位要轉 int: ['pricingInfo_isAccessTP', 'legs0_segments0_baggageAllowance_quantity', 'legs1_segments0_baggageAllowance_quantity', 'miniRules1_statusInfos', 'baggage_total', 'legs0_segments0_seatsAvailable', 'miniRules1_monetaryAmount']


In [2]:
exclude_cols = [
    'Id', 'ranker_id', 'selected'
]

feature_cols = [col for col in train_filled.columns if col not in exclude_cols]

print(f"Using {len(feature_cols)} features")

X = train_filled.select(feature_cols)
y = train_filled.select('selected')
groups = train_filled.select('ranker_id')

Using 80 features


In [3]:
import numpy as np
import polars as pl

# 確認這些物件都是Polars DataFrame
# X, y, groups
# 都是 shape [n_rows, n_cols]

# 先把 ranker_id轉list
unique_rankers = groups.select("ranker_id").unique().to_series().to_list()

# 打亂
np.random.seed(42)
np.random.shuffle(unique_rankers)

# 切8:2
n_train = int(0.8 * len(unique_rankers))
train_rankers = set(unique_rankers[:n_train])
val_rankers = set(unique_rankers[n_train:])

# 用 Polars 過濾 train/val
is_train = groups.select(pl.col("ranker_id").is_in(list(train_rankers)).alias("is_train"))

# 先 concat mask
X_with_mask = X.with_columns(is_train)
y_with_mask = y.with_columns(is_train)
groups_with_mask = groups.with_columns(is_train)

# 分割 DataFrame
X_train_df = X_with_mask.filter(pl.col("is_train"))
X_val_df = X_with_mask.filter(~pl.col("is_train"))
y_train_df = y_with_mask.filter(pl.col("is_train"))
y_val_df = y_with_mask.filter(~pl.col("is_train"))
groups_train_df = groups_with_mask.filter(pl.col("is_train"))
groups_val_df = groups_with_mask.filter(~pl.col("is_train"))

# 再轉 numpy (分批)
X_train_np = X_train_df.drop("is_train").to_numpy()
X_val_np = X_val_df.drop("is_train").to_numpy()
y_train_np = y_train_df.drop("is_train").to_numpy().flatten()
y_val_np = y_val_df.drop("is_train").to_numpy().flatten()
groups_train_np = groups_train_df.drop("is_train").to_numpy().flatten()
groups_val_np = groups_val_df.drop("is_train").to_numpy().flatten()
# 最後計算 group sizes
group_sizes_train = (
    groups_train_df.drop("is_train")
    .group_by("ranker_id", maintain_order=True)
    .agg(pl.len())['len']
    .to_numpy()
)

group_sizes_val = (
    groups_val_df.drop("is_train")
    .group_by("ranker_id", maintain_order=True)
    .agg(pl.len())['len']
    .to_numpy()
)


In [13]:
import lightgbm as lgb
import os

# 📌 參數
lgb_params = {
    "objective": "lambdarank",
    "metric": "ndcg",
    "ndcg_eval_at": [3],
    "max_depth": 10,
    "min_data_in_leaf": 10,      # ≈ min_child_weight
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "lambda_l2": 10.0,           # XGB 的 lambda
    "learning_rate": 0.05,
    "seed": 42,
    "num_threads": -1,
    "verbosity": 20
}

# 📌 建立 Dataset
dtrain = lgb.Dataset(
    X_train_np,
    label=y_train_np,
    feature_name = feature_cols,
    group=group_sizes_train
)
dval = lgb.Dataset(
    X_val_np,
    label=y_val_np,
    feature_name = feature_cols,
    group=group_sizes_val,
    reference=dtrain
)

# 📌 訓練
lgb_model = lgb.train(
    lgb_params,
    dtrain,
    num_boost_round=500,
    valid_sets=[dtrain, dval],
    valid_names=["train", "val"],
)

# 📌 儲存模型
model_path = os.path.join(save_dir, f"top{n_top}")
os.makedirs(model_path, exist_ok=True)
model_file = os.path.join(model_path, "lgb_ranker_model.txt")

lgb_model.save_model(model_file)
print(f"✅ 已儲存模型到 'lgb_ranker_model.txt' in {model_file}")


[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=100) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=1267650600228229401496703205376) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=100) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=1267650600228229401496703205376) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.817410
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.210568
[LightGBM] [Debug] init for col-wise cost 0.217633 seconds, init for row-wise cost 0.682065 seconds
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testi

In [17]:
import polars as pl
from scripts.hitrate import compute_hitrate_at_3
# 預測
val_preds = lgb_model.predict(X_val_np)

# 計算 HitRate
compute_hitrate_at_3(groups_val_np, y_val_np, val_preds)
import pandas as pd
import os

# 三種重要性對應 LightGBM
importance_types = {
    "weight": "split",    # 等於 XGBoost 的 "weight"
    "gain": "gain",       # 等於 XGBoost 的 "gain"
    # LightGBM 沒有 "cover"，只有 split/gain
}

importance_all = {}

for k, lgb_type in importance_types.items():
    imp_raw = lgb_model.feature_importance(importance_type=lgb_type)
    imp_named = dict(zip(lgb_model.feature_name(), imp_raw))
    sorted_imp = sorted(imp_named.items(), key=lambda x: x[1], reverse=True)
    importance_all[k] = sorted_imp

# weight榜
df_weight = pd.DataFrame(importance_all["weight"], columns=["feature", "weight_rank"])
df_weight["weight_rank_pos"] = df_weight.index

# gain榜
df_gain = pd.DataFrame(importance_all["gain"], columns=["feature", "gain_rank"])
df_gain["gain_rank_pos"] = df_gain.index

# 合併
df_merged = (
    df_weight
    .merge(df_gain, on="feature", how="outer")
)

# 補空值
df_merged["weight_rank_pos"] = df_merged["weight_rank_pos"].fillna(9999)
df_merged["gain_rank_pos"] = df_merged["gain_rank_pos"].fillna(9999)

# 最小rank
df_merged["min_rank"] = df_merged[["weight_rank_pos", "gain_rank_pos"]].min(axis=1)

# 排序
df_merged_sorted = df_merged.sort_values("min_rank")

# Top50
top50 = df_merged_sorted.head(50)

# 顯示
print(top50[["feature", "weight_rank", "gain_rank"]])

# 輸出CSV
csv_path = os.path.join(save_dir, f"top{n_top}/lgb_feature_importance.csv")
df_merged_sorted.to_csv(csv_path, index=False)
print(f"✅ 已輸出 feature_importance.csv 到 {csv_path}")


✅ HitRate@3 (groups size in [10, inf]): 0.4376
                         feature  weight_rank      gain_rank
13                 price_per_fee         1779  162549.558062
18        total_is_min_transfers           73  484589.317835
19      total_num_transfers_rank          103  412727.427146
11            price_per_duration         1572  132229.592556
8   legs0_segments0_flightNumber         1487   86169.376355
15                 price_per_tax         1469   77070.443410
9       price_from_median_zscore          886  121969.296433
1                      companyID         1321   55707.555080
16              price_percentile         1122  105525.248924
12       price_per_duration_rank          888   67554.778963
4           legs0_arrivalAt_hour          786   43623.019606
5         legs0_departureAt_hour          766   34222.141737
14            price_per_fee_rank          667   42109.144300
2          days_before_departure          657   21710.026397
10          price_minus_fee_rank      

In [4]:
import gc
del train_filled, X, X_train_df,X_val_df,X_with_mask, y, y_train_df,y_val_df,y_with_mask,
gc.collect()

0

In [5]:
import gc
del groups, groups_train_df, groups_train_np, groups_val_df, is_train, groups_with_mask
gc.collect()

0

In [6]:
import xgboost as xgb
import gc

# DMatrix 建立 (不再用 X.columns)
dtrain = xgb.DMatrix(
    X_train_np,
    label=y_train_np,
    feature_names = feature_cols,
)
dtrain.set_group(group_sizes_train)
del X_train_np, y_train_np, group_sizes_train
gc.collect()
dval = xgb.DMatrix(
    X_val_np,
    label=y_val_np,
    feature_names = feature_cols,
)
dval.set_group(group_sizes_val)
del X_val_np, group_sizes_val
gc.collect()

7

In [7]:
import json
import os
# 參數
model_dir = os.path.join(save_dir, f"top{n_top}")
os.makedirs(model_dir, exist_ok=True)


xgb_params = {
    'objective': 'rank:pairwise',
    'eval_metric': 'ndcg@3',
    'max_depth': 11,
    'min_child_weight': 1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'lambda': 10.0,
    'learning_rate': 0.05,
    'seed': 42,
    'n_jobs': -1,
}

# 儲存成JSON
params_path = os.path.join(model_dir, "xgb_params.json")

with open(params_path, "w") as f:
    json.dump(xgb_params, f, indent=4)

print(f"✅ 參數已存檔: {params_path}")


# 訓練
xgb_model = xgb.train(
    xgb_params,
    dtrain,
    num_boost_round=500,
    evals=[(dtrain, "train"), (dval, "val")],
    early_stopping_rounds=50,
    verbose_eval=20
)
# 存成二進位模型
model_path = os.path.join(save_dir, f"top{n_top}/xgb_ranker_model.bin")

xgb_model.save_model(model_path)
print(f"✅ 已儲存模型到 'xgb_ranker_model.bin' in {model_path}")



✅ 參數已存檔: model_output/selected_features_xgb/one_model/features_v1_with_view/top80/xgb_params.json
[0]	train-ndcg@3:0.35153	val-ndcg@3:0.34476
[20]	train-ndcg@3:0.47988	val-ndcg@3:0.43547
[40]	train-ndcg@3:0.51050	val-ndcg@3:0.44910
[60]	train-ndcg@3:0.53173	val-ndcg@3:0.45799
[80]	train-ndcg@3:0.54920	val-ndcg@3:0.46455
[100]	train-ndcg@3:0.56478	val-ndcg@3:0.47007
[120]	train-ndcg@3:0.57753	val-ndcg@3:0.47364
[140]	train-ndcg@3:0.58856	val-ndcg@3:0.47806
[160]	train-ndcg@3:0.59911	val-ndcg@3:0.48068
[180]	train-ndcg@3:0.61070	val-ndcg@3:0.48530
[200]	train-ndcg@3:0.62132	val-ndcg@3:0.48712
[220]	train-ndcg@3:0.62999	val-ndcg@3:0.48910
[240]	train-ndcg@3:0.63863	val-ndcg@3:0.49114
[260]	train-ndcg@3:0.64703	val-ndcg@3:0.49262
[280]	train-ndcg@3:0.65447	val-ndcg@3:0.49451
[300]	train-ndcg@3:0.66207	val-ndcg@3:0.49536
[320]	train-ndcg@3:0.66875	val-ndcg@3:0.49766
[340]	train-ndcg@3:0.67617	val-ndcg@3:0.50002
[360]	train-ndcg@3:0.68418	val-ndcg@3:0.50167
[380]	train-ndcg@3:0.69321	val-ndc

/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_69832/3249297446.py:42: UserWarning: [05:10:49] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1748292887431/work/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_path)


In [ ]:
import polars as pl
from scripts.hitrate import compute_hitrate_at_3
# 預測
val_preds = xgb_model.predict(dval)


# 計算 HitRate
hitrate = compute_hitrate_at_3(groups_val_np, y_val_np, val_preds)
hitrate_records = []
hitrate_records.append({"split_label": "overall", "hitrate": hitrate})


hitrate_df = pl.DataFrame(hitrate_records)
csv_path = os.path.join(model_dir, "hitrate_summary.csv")
hitrate_df.write_csv(csv_path)
print(f"\n✅ 已儲存所有 Hitrate 結果至 {csv_path}")



✅ HitRate@3 (groups size in [1, inf]): 0.3858

✅ 已儲存所有 Hitrate 結果至 model_output/selected_features_xgb/one_model/features_v1_with_view/top80/hitrate_summary.csv


In [9]:
import pandas as pd

# 先準備特徵名稱
feature_names = feature_cols

# 取三種重要性
importance_types = ["weight", "gain", "cover"]
importance_all = {}

for imp_type in importance_types:
    imp_raw = xgb_model.get_score(importance_type=imp_type)
    imp_named = {}
    for k, v in imp_raw.items():
        imp_named[k] = v
    # 排序
    sorted_imp = sorted(imp_named.items(), key=lambda x: x[1], reverse=True)
    importance_all[imp_type] = sorted_imp

# 把三個榜單放成DataFrame方便比對
df_weight = pd.DataFrame(importance_all["weight"], columns=["feature", "weight_rank"])
df_weight["weight_rank_pos"] = df_weight.index

df_gain = pd.DataFrame(importance_all["gain"], columns=["feature", "gain_rank"])
df_gain["gain_rank_pos"] = df_gain.index

df_cover = pd.DataFrame(importance_all["cover"], columns=["feature", "cover_rank"])
df_cover["cover_rank_pos"] = df_cover.index

# 合併
df_merged = (
    df_weight
    .merge(df_gain, on="feature", how="outer")
    .merge(df_cover, on="feature", how="outer")
)

# 把不存在的rank補大數字
df_merged["weight_rank_pos"] = df_merged["weight_rank_pos"].fillna(9999)
df_merged["gain_rank_pos"] = df_merged["gain_rank_pos"].fillna(9999)
df_merged["cover_rank_pos"] = df_merged["cover_rank_pos"].fillna(9999)

# 計算「三個榜單中最早出現的位置」
df_merged["min_rank"] = df_merged[["weight_rank_pos", "gain_rank_pos", "cover_rank_pos"]].min(axis=1)

# 排序
df_merged_sorted = df_merged.sort_values("min_rank")

# 取前50
top50 = df_merged_sorted.head(50)

# 顯示
print(top50[["feature", "weight_rank", "gain_rank", "cover_rank"]])
# 如果想輸出CSV
csv_path = os.path.join(model_dir, "feature_importance.csv")

df_merged_sorted.to_csv(csv_path, index=False)
print(f"✅ 已輸出{csv_path}")


                                        feature  weight_rank    gain_rank  \
6                                     companyID      15565.0     8.976178   
75                       total_is_min_transfers        249.0  2882.271240   
65                             price_percentile      15105.0    15.726101   
76                     total_num_transfers_rank        155.0   423.735016   
40                    legs0_weighted_mean_cabin        113.0    72.229271   
38  legs0_segments0_operatingCarrier_code_in_ff        678.0    13.394393   
57                     price_from_median_zscore      15076.0    14.224063   
66                       pricingInfo_isAccessTP        198.0    51.446644   
60                           price_per_duration      14914.0    12.420851   
61                      price_per_duration_rank      13801.0     8.399365   
11                                has_access_tp       1300.0    49.743481   
28                     legs0_max_duration_cabin         98.0    15.466170   

# Prediction

In [10]:
import os
import numpy as np
import polars as pl
import lightgbm as lgb

# 參數
model_path = "model_output/selected_features_xgb/one_model/top100/lgb_ranker_model.txt"
parquet_path = "data/test_filled.parquet"

# 讀取 LightGBM 模型
lgb_model = lgb.Booster(model_file=model_path)

# 取得 feature_names
model_features = lgb_model.feature_name()
if model_features is None or len(model_features) == 0:
    raise ValueError("❌ 模型沒有 feature_names，請確認訓練時有指定 feature_names")
print(f"✅ 模型共 {len(model_features)} 個features")

# 讀取 test_filled
df = pl.read_parquet(parquet_path)
print(f"✅ 讀取 test_filled，共 {df.height} rows")

# 檢查缺失
missing_in_data = [f for f in model_features if f not in df.columns]
if missing_in_data:
    raise ValueError(f"❌ 下列特徵在 test_filled 不存在: {missing_in_data}")

# 篩選&排序
df_for_predict = df.select(model_features)
X_np = df_for_predict.to_numpy()

# 預測
preds = lgb_model.predict(X_np)
print(f"✅ 預測完成，共 {len(preds)} 筆")

# 回存結果
df_result = (
    df
    .with_columns([
        pl.Series("selected", preds)
    ])
)

# 查看前幾筆
print(df_result.head())


[LightGBM] [Fatal] Could not open model_output/selected_features_xgb/one_model/top100/lgb_ranker_model.txt


LightGBMError: Could not open model_output/selected_features_xgb/one_model/top100/lgb_ranker_model.txt

In [10]:
import os
import numpy as np
import polars as pl
import xgboost as xgb

# 參數
model_path = "model_output/selected_features_xgb/one_model/features_v1_with_view/top80/xgb_ranker_model.bin"
parquet_path = "data/test_filled_v1_with_veiw.parquet"

# 讀取模型
xgb_model = xgb.Booster(model_file=model_path)

# 確認 feature_names
model_features = xgb_model.feature_names
if model_features is None:
    raise ValueError("❌ 模型沒有 feature_names，請確認訓練時有指定 feature_names")
print(f"✅ 模型共 {len(model_features)} 個features")

# 讀取 test_filled
df = pl.read_parquet(parquet_path)
print(f"✅ 讀取 test_filled，共 {df.height} rows")

# 檢查缺失
missing_in_data = [f for f in model_features if f not in df.columns]
if missing_in_data:
    raise ValueError(f"❌ 下列特徵在 test_filled 不存在: {missing_in_data}")

# 篩選&排序
df_for_predict = df.select(model_features)
X_np = df_for_predict.to_numpy()

# 預測
dtest = xgb.DMatrix(X_np, feature_names=model_features)
preds = xgb_model.predict(dtest)
print(f"✅ 預測完成，共 {len(preds)} 筆")

# 回存結果
df_result = (
    df
    .with_columns([
        pl.Series("selected", preds)
    ])
)

# 查看前幾筆
print(df_result.head())


✅ 模型共 80 個features
✅ 讀取 test_filled，共 6897776 rows
✅ 預測完成，共 6897776 筆
shape: (5, 264)
┌──────────┬────────┬───────────┬────────────┬───┬────────────┬────────────┬───────────┬───────────┐
│ Id       ┆ bySelf ┆ companyID ┆ nationalit ┆ … ┆ legs1_segm ┆ legs1_segm ┆ legs1_seg ┆ selected  │
│ ---      ┆ ---    ┆ ---       ┆ y          ┆   ┆ ents1_key_ ┆ ents2_key_ ┆ ments3_ke ┆ ---       │
│ i64      ┆ i8     ┆ i64       ┆ ---        ┆   ┆ view_count ┆ view_count ┆ y_view_co ┆ f32       │
│          ┆        ┆           ┆ i64        ┆   ┆ …          ┆ …          ┆ unt…      ┆           │
│          ┆        ┆           ┆            ┆   ┆ ---        ┆ ---        ┆ ---       ┆           │
│          ┆        ┆           ┆            ┆   ┆ u32        ┆ u32        ┆ u32       ┆           │
╞══════════╪════════╪═══════════╪════════════╪═══╪════════════╪════════════╪═══════════╪═══════════╡
│ 18144679 ┆ 1      ┆ 62840     ┆ 36         ┆ … ┆ 1          ┆ 1          ┆ 1         ┆ -0.958247 │
│ 181

In [11]:
import os
import polars as pl

def export_submission_parquets(
    test_filled_with_preds: pl.DataFrame,
    output_dir: str,
    raw_filename: str = "raw_submission.parquet",
    ranked_filename: str = "rank_submission.parquet"
):
    """
    根據 test_filled_with_preds 輸出兩個 parquet:
    1. 原始分數 (selected)
    2. rank 排序 (selected)
    """
    # 檢查目錄
    os.makedirs(output_dir, exist_ok=True)

    # Subset + __index_level_0__
    subset_df = (
        test_filled_with_preds
        .select(["Id", "ranker_id", "selected"])
        .with_columns(
            pl.col("Id").alias("__index_level_0__")
        )
        .with_columns([
            pl.col("Id").cast(pl.Int64),
            pl.col("ranker_id").cast(pl.Utf8),
            pl.col("selected").cast(pl.Float64),
            pl.col("__index_level_0__").cast(pl.Int64)
        ])
    )

    # 儲存原始 parquet
    raw_path = os.path.join(output_dir, raw_filename)
    subset_df.write_parquet(raw_path)
    print(f"✅ 已儲存原始 submission: {raw_path}")
    print(subset_df)

    # Rank 排名
    ranked_df = subset_df.with_columns(
        pl.col("selected")
          .rank(method="ordinal", descending=True)
          .over("ranker_id")
          .alias("selected")
    )

    # 儲存排名 parquet
    ranked_path = os.path.join(output_dir, ranked_filename)
    ranked_df.write_parquet(ranked_path)
    print(f"✅ 已儲存rank submission: {ranked_path}")
    print(ranked_df)
    
n_top = 80
export_submission_parquets(
    test_filled_with_preds=df_result,   # 你的帶有 selected 分數的 DataFrame
    output_dir=f"model_output/selected_features_xgb/one_model/features_v1_with_view/top{n_top}",
    ranked_filename = "rank_submission.parquet",
    raw_filename ="raw_submission.parquet",
)


✅ 已儲存原始 submission: model_output/selected_features_xgb/one_model/features_v1_with_view/top80/raw_submission.parquet
shape: (6_897_776, 4)
┌──────────┬─────────────────────────────────┬───────────┬───────────────────┐
│ Id       ┆ ranker_id                       ┆ selected  ┆ __index_level_0__ │
│ ---      ┆ ---                             ┆ ---       ┆ ---               │
│ i64      ┆ str                             ┆ f64       ┆ i64               │
╞══════════╪═════════════════════════════════╪═══════════╪═══════════════════╡
│ 18144679 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ -0.958247 ┆ 18144679          │
│ 18144680 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ -0.528382 ┆ 18144680          │
│ 18144681 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ -1.854798 ┆ 18144681          │
│ 18144682 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ -0.814971 ┆ 18144682          │
│ 18144683 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ -0.461108 ┆ 18144683          │
│ …        ┆ …                               ┆ …         ┆ …            

In [ ]:
v1:
/v1_model/raw_submission.parquet",
/v2_model/raw_submission.parquet",
/v2_1/raw_submission.parquet",
/v2_2/raw_submission.parquet",
/top60/raw_submission.parquet",
    
v2:
/v1_model/raw_submission.parquet",
/top120/raw_submission.parquet",
/top100/raw_submission.parquet",
/top80/raw_submission.parquet",
/top60/raw_submission.parquet",
    
/v1_model/raw_submission.parquet",
/top100/raw_submission.parquet"

In [4]:
from scripts.result_esemble import compute_ranked_average_selected_from_submissions

submission_paths = [
        "model_output/all_features_xgb/v1/v1_model/raw_submission.parquet",
        "model_output/all_features_xgb/v1/v2_model/raw_submission.parquet", #0.486
        "model_output/all_features_xgb/v1/v2_1/raw_submission.parquet", #0.492
        "model_output/all_features_xgb/v1/v2_2/raw_submission.parquet",
        "model_output/selected_features_xgb/one_model/features_v1/top60/raw_submission.parquet", # 0.49
        "model_output/selected_features_xgb/one_model/features_v1/top80/raw_submission.parquet",#0.49
        "model_output/selected_features_xgb/one_model/features_v1/top100/raw_submission.parquet",
        "model_output/selected_features_xgb/one_model/features_v1/top120/raw_submission.parquet",
        
        "model_output/all_features_xgb/v2/v1_model/raw_submission.parquet", #0.4912
        'model_output/selected_features_xgb/one_model/features_v2/top120/raw_submission.parquet', # 0.4915
        "model_output/selected_features_xgb/one_model/features_v2/top100/raw_submission.parquet", #0.486
        "model_output/selected_features_xgb/one_model/features_v2/top80/raw_submission.parquet",
        # "model_output/selected_features_xgb/one_model/features_v2/top60/raw_submission.parquet", 0.476
        
        "model_output/all_features_xgb/v3/v1_model/raw_submission.parquet",
        "model_output/selected_features_xgb/one_model/features_v3/top100/raw_submission.parquet", #0.479
        "model_output/selected_features_xgb/one_model/features_v3/top80/raw_submission.parquet", #0.484
        
        
        "model_output/selected_features_xgb/one_model/features_v1_with_view/top100/raw_submission.parquet",
        "model_output/selected_features_xgb/one_model/features_v1_with_view/top120/raw_submission.parquet",
        "model_output/all_features_xgb/v1_add_flight_view/v1_model/raw_submission.parquet"
]

avg_df = compute_ranked_average_selected_from_submissions(submission_paths)

# 顯示前5筆
print(avg_df.head())

# 如需輸出
avg_df.write_parquet("submission_selected_mean.parquet")
print("✅ 已輸出平均submission")


✅ 讀取: model_output/all_features_xgb/v1/v1_model/raw_submission.parquet
✅ 讀取: model_output/all_features_xgb/v1/v2_model/raw_submission.parquet
✅ 讀取: model_output/all_features_xgb/v1/v2_1/raw_submission.parquet
✅ 讀取: model_output/all_features_xgb/v1/v2_2/raw_submission.parquet
✅ 讀取: model_output/selected_features_xgb/one_model/features_v1/top60/raw_submission.parquet
✅ 讀取: model_output/selected_features_xgb/one_model/features_v1/top80/raw_submission.parquet
✅ 讀取: model_output/selected_features_xgb/one_model/features_v1/top100/raw_submission.parquet
✅ 讀取: model_output/selected_features_xgb/one_model/features_v1/top120/raw_submission.parquet
✅ 讀取: model_output/all_features_xgb/v2/v1_model/raw_submission.parquet
✅ 讀取: model_output/selected_features_xgb/one_model/features_v2/top120/raw_submission.parquet
✅ 讀取: model_output/selected_features_xgb/one_model/features_v2/top100/raw_submission.parquet
✅ 讀取: model_output/selected_features_xgb/one_model/features_v2/top80/raw_submission.parquet
✅ 讀取:

In [ ]:
avg_df

Id,ranker_id,selected,__index_level_0__
i64,str,u32,i64
18144679,"""c9373e5f772e43d593dd6ad2fa90f6…",82,18144679
18144680,"""c9373e5f772e43d593dd6ad2fa90f6…",25,18144680
18144681,"""c9373e5f772e43d593dd6ad2fa90f6…",224,18144681
18144682,"""c9373e5f772e43d593dd6ad2fa90f6…",114,18144682
18144683,"""c9373e5f772e43d593dd6ad2fa90f6…",49,18144683
…,…,…,…
25043143,"""c5622e0de0594bde95a4dd8c1fcff7…",9,25043143
25043144,"""c5622e0de0594bde95a4dd8c1fcff7…",1,25043144
25043145,"""c5622e0de0594bde95a4dd8c1fcff7…",11,25043145


: 